- В коде из методички реализуйте один или несколько из критериев останова (количество листьев, количество используемых признаков, глубина дерева и т.д.).
- Для задачи классификации обучить дерево решений с использованием критериев разбиения Джини и Энтропия. Сравнить качество классификации, сделать выводы. 
- 3*. Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.

In [601]:
import matplotlib.pyplot as plt
import random

from matplotlib.colors import ListedColormap
from sklearn import datasets

import numpy as np
import math

In [603]:
n_= int(input('Введите кол-во фичей (от 2 до 5): '))

Введите кол-во фичей (от 2 до 5): 3


In [604]:
leaf_= int(input('Введите минимальное кол-во листьев (от 2 до 10): '))

Введите минимальное кол-во листьев (от 2 до 10): 5


In [605]:
# сгенерируем данные
classification_data, classification_labels = datasets.make_classification(n_features=n_, n_informative = 2, 
                                                      n_classes = 2, n_redundant=0, 
                                                      n_clusters_per_class=1, random_state=33)

In [606]:
classification_labels

array([0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0])

In [607]:
classification_data.shape[0]

100

In [608]:
classification_data.shape[1]

3

In [609]:
# Реализуем класс узла
class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога, разделяющего дерево на true и false
        self.true_branch = true_branch  # ветка (стебель) поддерева, удовлетворяющее условию в узле
        self.false_branch = false_branch  # ветка (стебель) поддерева, не удовлетворяющее условию в узле

In [610]:
# И класс верхушечного терминального узла (конечного листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction  

In [739]:
# Расчет критериальный признаков

def info_criteria(type_criteria:str, labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    info_criteria = []
    
    if type_criteria == 'gini':
    
    #  расчет критерия
        impurity = 1
        for label in classes:
            p = classes[label] / len(labels)
            impurity -= p ** 2
        
        info_criteria.append(impurity)
    
    elif type_criteria == 'shenon': 
        
        entropy=0
        for i in classes: 
            p = classes[label]/len(labels)
            entropy -= p*np.math.log2(p)
        
        info_criteria.append(entropy)
        
    return info_criteria[0]

In [613]:
info_criteria('shenon', classification_labels)

0.9908594647592938

In [614]:
info_criteria('gini', classification_labels)

0.4998

In [740]:
# Расчет качества по энтропии Шэнона

def quality_shenon_entropy(left_labels, right_labels, current_shenon):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_shenon - (-p) * info_criteria('shenon', left_labels) - (1 - p) * info_criteria('shenon', right_labels)

In [741]:
# Расчет качества по Джини

def quality_gini(left_labels, right_labels, current_gini):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_gini - (-p) * info_criteria('gini', left_labels) - (1 - p) * info_criteria('gini', right_labels)

In [742]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [743]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels, type_criteria, leaf=leaf_):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = leaf_
    
    if type_criteria == 'gini':
        current_gini = info_criteria('gini', labels)
    else:
        current_shenon = info_criteria('shenon', labels)

    best_quality = 0   #начальный параметр качества, равен нулю
    best_t = None      #начальный параметр, порог
    best_index = None  #начальный параметр, индекс фичи
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            if type_criteria == 'gini': 
                current_quality = quality_gini(true_labels, false_labels, current_gini)
                
            elif type_criteria == 'shenon':  
                current_quality = quality_shenon_entropy(true_labels, false_labels, current_shenon)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [626]:
#может быть закинуть quality в массив и поработать так с ним??? 
# как реализовать ввод листьев через input... Где-то был пример. Надо вспоминать. 

In [744]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, type_criteria, leaf=leaf_):

    quality, t, index = find_best_split(data, labels, type_criteria, leaf=leaf_)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, type_criteria, leaf=leaf_)
    false_branch = build_tree(false_data, false_labels, type_criteria, leaf=leaf_)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [745]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [746]:
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [747]:
# Разобьем выборку на обучающую и тестовую

from sklearn import model_selection

train_data, test_data, train_labels, test_labels = model_selection.train_test_split(classification_data, 
                                                                                     classification_labels, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)

In [748]:
# Построим дерево по обучающей выборке
my_tree = build_tree(train_data, train_labels, type_criteria='shenon', leaf=leaf_)

In [749]:
# Напечатаем ход нашего дерева
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index))
    print(spacing + 'Порог', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")
    
print_tree(my_tree)

Индекс 0
Порог 2.0101109109981
--> True:
  Индекс 0
  Порог 1.7160256769490312
  --> True:
    Индекс 0
    Порог 1.321702071253235
    --> True:
      Индекс 1
      Порог 2.083493567759761
      --> True:
        Индекс 1
        Порог 1.838978814328743
        --> True:
          Индекс 1
          Порог 1.2447102045271525
          --> True:
            Индекс 0
            Порог 0.8274884684142032
            --> True:
              Индекс 1
              Порог 0.9155124476066724
              --> True:
                Индекс 0
                Порог 0.5518915729504906
                --> True:
                  Индекс 2
                  Порог 0.47306489847333005
                  --> True:
                    Индекс 1
                    Порог 0.30427455266022674
                    --> True:
                      Индекс 0
                      Порог 0.455441823029495
                      --> True:
                        Индекс 0
                        Порог -0.112633570652950

In [750]:
 # Получим ответы для обучающей выборки 
train_answers = predict(train_data, my_tree)

In [751]:
# И получим ответы для тестовой выборки
answers = predict(test_data, my_tree)

In [752]:
# Введем функцию подсчета точности как доли правильных ответов
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [753]:
# Точность на обучающей выборке
train_accuracy = accuracy_metric(train_labels, train_answers)
train_accuracy

91.42857142857143

In [754]:
# Точность на тестовой выборке
test_accuracy = accuracy_metric(test_labels, answers)
test_accuracy

96.66666666666667

In [755]:
my_tree_gini = build_tree(train_data, train_labels, type_criteria='gini', leaf=leaf_)

In [756]:
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index))
    print(spacing + 'Порог', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")
    
print_tree(my_tree_gini)

Индекс 0
Порог 2.0101109109981
--> True:
  Индекс 1
  Порог 2.083493567759761
  --> True:
    Индекс 0
    Порог 1.7160256769490312
    --> True:
      Индекс 1
      Порог 1.838978814328743
      --> True:
        Индекс 0
        Порог 1.321702071253235
        --> True:
          Индекс 1
          Порог 1.2447102045271525
          --> True:
            Индекс 0
            Порог 0.8274884684142032
            --> True:
              Индекс 1
              Порог 0.9155124476066724
              --> True:
                Индекс 0
                Порог 0.5518915729504906
                --> True:
                  Индекс 2
                  Порог 0.47306489847333005
                  --> True:
                    Индекс 0
                    Порог 0.455441823029495
                    --> True:
                      Индекс 1
                      Порог 0.30427455266022674
                      --> True:
                        Индекс 0
                        Порог -0.112633570652950

In [757]:
 # Получим ответы для обучающей выборки 
train_answers_gini = predict(train_data, my_tree_gini)

In [758]:
# И получим ответы для тестовой выборки
answers_gini = predict(test_data, my_tree_gini)

In [759]:
# Введем функцию подсчета точности как доли правильных ответов
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [760]:
# Точность на обучающей выборке
train_accuracy_gini = accuracy_metric(train_labels, train_answers_gini)
train_accuracy

91.42857142857143

In [761]:
# Точность на тестовой выборке
test_accuracy = accuracy_metric(test_labels, answers_gini)
test_accuracy

96.66666666666667

Заключение о проделанной работе: 
    - я полагаю, что это не лучшая реализация алгоритма. Наверное, тут можно оперировать ООП и отдельно определить по какому критерию информативности будем определять качество и какое минимальное кол-во листьев можно выбрать 
    - очень долго не мог понять, почему train и test accuracy было такое низкое. В начальных параметрах выбрал кол-во классов 3. Невнимательность - наше все. 
    - я реализовал два дерева: в одном использовал критерий Джини, во втором - Шэнон. Почему-то качество вышло одно и тоже :(. Где-то закралась ошибка. Но функционально реализовано вроде все верно, поэтому сам идентифицировать не могу. 
    - в разборе интересно посмотреть как реализовано третье ДЗ и конечно, более высокоуровневое ООП. Подозреваю, что именно так было выполнено образцовое решение, ибо Наставник спрашивал, как у нас с ООП